In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from typing import Dict, List
import matplotlib.pyplot as plt

def plot(metric_results: Dict[str, List[float]] , metric_name: str) -> None:
  plt.plot(metric_results[metric_name])
  plt.ylabel(metric_name)
  plt.xlabel("Step")
  plt.title("{} versus Step".format(metric_name))

In [ ]:
# Set hyperparameters.
BATCH_SIZE = 8
TRAINING_LOOPS = 20
STEPS_PER_LOOP = 2

# Set  simulation environment parameters.
RANK_K = 20
NUM_ACTIONS = 2
PER_ARM = False

# Set agent parameters.
TIKHONOV_WEIGHT = 0.001
AGENT_ALPHA = 10.0

In [ ]:
from collections import defaultdict
import functools
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from typing import Callable, Dict, List, Any, Optional, TypeVar
from pathlib import Path
import pandas as pd

In [ ]:
!pip install tf_agents -q

In [ ]:
import tensorflow as tf
import tf_agents
from tf_agents.agents import TFAgent
from tf_agents.bandits.agents import lin_ucb_agent
from tf_agents.bandits.agents.examples.v2 import trainer
from tf_agents.bandits.environments import environment_utilities
from tf_agents.bandits.environments import movielens_py_environment
from tf_agents.bandits.metrics import tf_metrics as tf_bandit_metrics
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import TFEnvironment
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.metrics.tf_metric import TFStepMetric
from tf_agents.policies import policy_saver

In [ ]:
T = TypeVar("T")

In [ ]:
data_dir='/content/drive/MyDrive/RL_recommendation-zaid/DATA/Research dataset_2.xlsx'

In [ ]:
#read the xlsx file
df = pd.read_excel(data_dir)
df.head()

,username,age,sex,location,Relationship Status,designation,salary,likes,dislikes,strengths,weaknesses,negative state of emotion,reason
0,NirmalS,35,M,Colombo,In a relationship,Software Engineer,75000,Reading,Traffic,Problem-solving,Procrastination,Frustration,Long work hours
1,PriyankaG,28,F,Gampaha,Married,Teacher,45000,Gardening,Crowds,Patience,Public Speaking,Anxiety,teaching new class of students
2,RavindraP,42,M,Kandy,Mother,Manager,90000,Traveling,Gossip,Leadership,Time Management,Stress,managing team and deliver results
3,DilinaK,31,F,Galle,Single,Nurse,55000,Cooking,Rudeness,Empathy,Organization,Sadness,no partner
4,SharanC,26,M,Jaffna,Single,Accountant,65000,Music,Dishonesty,Attention to detail,Confidence,Insecurity,not in perfect body shape


In [ ]:
#remove all columns except username, age, sex, Relationship Status, negative state of emotion
df_new = df[["age", "sex", "Relationship Status", "negative state of emotion", 'reason']]
df_new.head()

,age,sex,Relationship Status,negative state of emotion,reason
0,35,M,In a relationship,Frustration,Long work hours
1,28,F,Married,Anxiety,teaching new class of students
2,42,M,Mother,Stress,managing team and deliver results
3,31,F,Single,Sadness,no partner
4,26,M,Single,Insecurity,not in perfect body shape


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

columns_to_encode = ['sex','Relationship Status', 'negative state of emotion','reason']

for column in columns_to_encode:
    if df_new[column].dtype == 'object':
        df_new[column] = label_encoder.fit_transform(df_new[column])

df_new.head()

<ipython-input-149-23b868680e4d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[column] = label_encoder.fit_transform(df_new[column])
<ipython-input-149-23b868680e4d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[column] = label_encoder.fit_transform(df_new[column])
<ipython-input-149-23b868680e4d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

,age,sex,Relationship Status,negative state of emotion,reason
0,35,2,17,53,121
1,28,0,18,5,264
2,42,2,25,119,253
3,31,0,29,107,255
4,26,2,29,66,259


### Environment Setup

In [ ]:
import numpy as np
import tensorflow as tf

from tf_agents.environments import py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts
from tf_agents.agents.dqn import dqn_agent
from tf_agents.networks import q_network
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.metrics import tf_metrics
from tf_agents.drivers import dynamic_step_driver

In [ ]:
class UserEnvironment(py_environment.PyEnvironment):
    def __init__(self, data):
        self._data = data
        self._index = 0  # To keep track of which row we're on

        self._action_spec = array_spec.BoundedArraySpec(
            shape=(), dtype=np.int32, minimum=0, maximum=NUM_ACTIONS - 1, name='action')
        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(4,), dtype=np.int32, minimum=0, name='observation')  # age, sex, relationship status, emotional state

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        self._index = 0  # Reset to the start of the data
        self._state = self._get_state()
        self._episode_ended = False
        return ts.restart(self._state)

    def _step(self, action):
        if self._episode_ended:

            return self.reset()


        if action == 0:
            self._episode_ended = True
        elif action == 1:
            self._episode_ended = False
        else:
            raise ValueError('`action` should be 0 or 1.')

        if self._episode_ended:
            reward = 0.0
            return ts.termination(self._state, reward)
        else:
            self._state = self._get_state()
            reward = self._calculate_reward(action)
            return ts.transition(self._state, reward, discount=0.9)

    def _get_state(self):

        row = self._data.iloc[self._index]
        self._index += 1  # Move to the next row
        return np.array([row['age'], row['sex'], row['Relationship Status'], row['negative state of emotion']], dtype=np.int32)

    def _calculate_reward(self, action):
        correct_advice = self._data.iloc[self._index]['reason'] #reason means the correct advice here
        return 1.0 if action == correct_advice else -1.0

In [ ]:
from tf_agents.environments import tf_py_environment

def define_rl_environment() -> TFEnvironment:
    env = UserEnvironment(df_new)
    environment = tf_py_environment.TFPyEnvironment(env)
    return environment

In [ ]:
environment = define_rl_environment()
environment

In [ ]:
def define_rl_agent(environment: TFEnvironment) -> TFAgent:
    # Define RL agent/algorithm.
    agent = lin_ucb_agent.LinearUCBAgent(
        time_step_spec=environment.time_step_spec(),
        action_spec=environment.action_spec(),
        tikhonov_weight=TIKHONOV_WEIGHT,
        alpha=AGENT_ALPHA,
        dtype=tf.float32,
        accepts_per_arm_features=PER_ARM)
    return agent

In [ ]:
agent = define_rl_agent(environment)

In [ ]:
def compute_optimal_reward_with_custom_environment(time_step):
    # TODO
    optimal_reward = 1.0

    return optimal_reward


In [ ]:
def define_rl_metric(environment: TFEnvironment) -> List[Any]:
    optimal_reward_fn = compute_optimal_reward_with_custom_environment
    regret_metric = tf_bandit_metrics.RegretMetric(optimal_reward_fn)
    metrics = [regret_metric]
    return metrics

In [ ]:
metrics = define_rl_metric(environment)

In [ ]:
def train_agent(num_iterations, collect_steps_per_iteration, log_interval, eval_interval, environment, agent, replay_buffer, compute_avg_return, eval_env, num_eval_episodes):
    for _ in range(num_iterations):
        # Collect a few steps using collect_policy and save to the replay buffer.
        for _ in range(collect_steps_per_iteration):
            collect_step(environment, agent.collect_policy, replay_buffer)

        # Sample a batch of data from the buffer and update the agent's network.
        experience = replay_buffer.gather_all()
        train_loss = agent.train(experience)
        replay_buffer.clear()

        step = agent.train_step_counter.numpy()

        if step % log_interval == 0:
            print('step = {0}: loss = {1}'.format(step, train_loss.loss))

        if step % eval_interval == 0:
            avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
            print('step = {0}: Average Return = {1}'.format(step, avg_return))


In [ ]:
def train(root_dir: str, agent: TFAgent, environment: TFEnvironment, training_loops: int, steps_per_loop: int,
          additional_metrics: Optional[List[TFStepMetric]] = None, training_data_spec_transformation_fn: Optional[Callable[[T], T]] = None) -> Dict[str, List[float]]:

    if training_data_spec_transformation_fn is None:
        data_spec = agent.policy.trajectory_spec
    else:
        data_spec = training_data_spec_transformation_fn(agent.policy.trajectory_spec)

    replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
        data_spec=agent.collect_data_spec,
        batch_size=environment.batch_size,
        max_length=10
    )

    step_metric = tf_metrics.EnvironmentSteps()
    metrics = [
        tf_metrics.NumberOfEpisodes(),
        tf_metrics.AverageEpisodeLengthMetric(batch_size=environment.batch_size)
    ]
    if additional_metrics:
        metrics += additional_metrics

    if isinstance(environment.reward_spec(), dict):
        metrics += [tf_metrics.AverageReturnMultiMetric(
            reward_spec=environment.reward_spec(),
            batch_size=environment.batch_size)]
    else:
        metrics += [
            tf_metrics.AverageReturnMetric(batch_size=environment.batch_size)]

    metric_results = defaultdict(list)

    if training_data_spec_transformation_fn is not None:
        add_batch_fn = lambda data: replay_buffer.add_batch(
            training_data_spec_transformation_fn(data))
    else:
        add_batch_fn = replay_buffer.add_batch

    observers = [add_batch_fn, step_metric] + metrics

    driver = dynamic_step_driver.DynamicStepDriver(
        env=environment,
        policy=agent.collect_policy,
        num_steps=steps_per_loop * environment.batch_size,
        observers=observers)

    # Create a policy saver
    saver = policy_saver.PolicySaver(agent.policy)

    # Define the training loop
    for _ in range(training_loops):
        driver.run()
        for metric in metrics:
            metric.tf_summaries(train_step=step_metric.result())
            metric_results[type(metric).__name__].append(metric.result().numpy())
        saver.save(root_dir)

    return metric_results


In [ ]:
base_path = os.path.dirname(os.path.realpath('__file__'))
ROOT_PATH = base_path + "/artifacts"
PROFILER_PATH = base_path + "/profiler"

print("Root path - ",ROOT_PATH)
print("Profiler path - ",PROFILER_PATH)

Root path -  /content/artifacts
Profiler path -  /content/profiler


In [ ]:
tf.profiler.experimental.stop()

In [ ]:
tf.profiler.experimental.start('logdir_path')
metric_results = train(
    root_dir=ROOT_PATH,
    agent=agent,
    environment=environment,
    training_loops=TRAINING_LOOPS,
    steps_per_loop=STEPS_PER_LOOP,
    additional_metrics=metrics
    )
tf.profiler.experimental.stop()

/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.

## Evaluation

In [ ]:
plot(metric_results, "RegretMetric")

In [ ]:
plot(metric_results, "AverageReturnMetric")

## Predictions

In [ ]:
def predict_observations_by_users(observation: List[List[float]]) -> List[int]:
    # TODO
    trained_policy = tf.saved_model.load(ROOT_PATH)



    time_step = tf_agents.trajectories.restart(
        observation=observation,
        batch_size=tf.convert_to_tensor([BATCH_SIZE]),
    )
    action_step = trained_policy.action(time_step)
    return action_step.action.numpy().tolist()

In [ ]:
recommended_movie_ids = predict_observations_by_users(observation = [list(np.ones(20)) for _ in range(8)])
recommended_movie_ids